<a href="https://colab.research.google.com/github/AishaEvering/LLM_Mastery/blob/main/News_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 -qqq install datasets
!pip3 -qqq install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:
dataset = load_dataset('multi_news', split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for multi_news contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/multi_news.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['document', 'summary'],
    num_rows: 5622
})

In [ ]:
df = dataset.to_pandas().sample(2000, random_state=42)

In [ ]:
df.head()

,document,summary
4830,Tweet with a location \n \n You can add locati...,– Denis Finley has taken to Twitter to call Po...
1255,CNN host Piers Morgan just called to discuss h...,– CNN's Piers Morgan thinks gun-rights propone...
80,White House communications director Anthony Sc...,– New White House communications director Anth...
3044,CLOSE Scientists say they've found archaeologi...,– Scientists say they have the first physical ...
4486,Click image above to view graphic \n \n Althou...,– Scientists are calling it a breakthrough and...


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
model

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
passage_embeddings = list(model.encode(df['summary'].to_list()))

In [ ]:
passage_embeddings[0].shape

(384,)

In [ ]:
query = 'Find me some articles about technology and artificial intelligence.'

In [ ]:
query_embedding = model.encode(query)
query_embedding.shape

(384,)

In [ ]:
similarities = util.cos_sim(query_embedding, passage_embeddings)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/util.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  a = torch.tensor(a)


In [ ]:
top_indices = torch.topk(similarities.flatten(), k=3).indices
top_indices

tensor([1266, 1834, 1612])

In [ ]:
top_relevant_passages = [df.iloc[x.item()]['summary'][:200] + '...' for x in top_indices]
top_relevant_passages

['– Are you a "digital native" or a "digital immigrant," and does it make a difference? Research recently published in the Teaching and Teacher Education journal indicates the concept of so-called digit...',
 "– Using methods borrowed from Google, a group of researchers has analyzed all Wikipedia pages and determined that, at least on the English language version of the site, Frank Sinatra is the world's mo...",
 '– The "tech surge" to fix HealthCare.gov includes some names from the industry\'s biggest players. Among them, per a Health department blog post, is Michael Dickerson, on leave from his job as a site r...']

In [ ]:
def find_relevant_news(query:str):
  query_embedding = model.encode(query)

  similarities = util.cos_sim(query_embedding, passage_embeddings)

  top_indices = torch.topk(similarities.flatten(), k=3).indices

  top_relevant_passages = [df.iloc[x.item()]['summary'][:200] + '...' for x in top_indices]

  return top_relevant_passages

In [ ]:
find_relevant_news("Natural disasters")

['– The tsunami that killed hundreds, possibly thousands of people after an earthquake in Indonesia on Friday was much bigger and more devastating than would normally be expected after that kind of quak...',
 '– A sad milestone out of Japan: Two weeks after the quake struck, its official death toll has broken the 10,000 mark—and that number is still on the rise, with more than 17,400 missing. Police estimat...',
 '– When you live near a major dam, the last thing you want to hear is that the integrity of it has been "compromised" by landslides. But that\'s exactly what residents living below North Carolina\'s Lake...']

In [ ]:
find_relevant_news("Law enforcement and police")

['– The war of words between Chicago and the federal government over "sanctuary cities" policy is heating up. Attorney General Jeff Sessions slammed the city\'s leaders Monday over their lawsuit against ...',
 '– Greg Barnes was in a hurry to get home on Friday, so when he saw police lights behind him on State Road 332 in Muncie, Indiana, "immediately I knew I was in the wrong," he tells WISH. What followed ...',
 '– "We are not thugs. We are professionals," says the leader of a black policing group, addressing a speech in which President Trump urged officers to not be "too nice" when arresting suspects. Perry T...']

In [ ]:
find_relevant_news("Politics, diplomacy and nationalism")

['– Tunisia got a new interim president today—parliamentary leader Fouad Mebazaa—who is technically the third leader in 24 hours, notes the Los Angeles Times. He takes over from the previous interim pre...',
 '– The global relief effort in Haiti has devolved into a nasty power struggle, with countries and aid agencies furious at the US takeover of emergency operations. France, Brazil, which runs the UN peac...',
 '– Like a game of Risk, Russia and NATO have not only been suspiciously eying each other in recent months, but also conducting military exercises in border areas that the Washington Post says are makin...']